<a href="https://colab.research.google.com/github/rohithsing/SAP/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  # 1. Import Required Libraries
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
# 2. Load IMDB Dataset
# Only consider top 10,000 most frequent words
VOCAB_SIZE = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))

Training samples: 25000
Testing samples: 25000


In [ ]:
#3. Pad Sequences
# Maximum number of words per review
MAX_LEN = 200

X_train = pad_sequences(X_train, maxlen=MAX_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_LEN)

print("Input shape after padding:", X_train.shape)


Input shape after padding: (25000, 200)


In [ ]:
# 4. Build RNN Model
model = Sequential([
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=128,
        input_shape=(MAX_LEN,)   # <-- THIS fixes summary
    ),
    SimpleRNN(128),
    Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 5. Compile Model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"])


In [ ]:
# 6. Train Model
history = model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.5253 - loss: 0.6932 - val_accuracy: 0.5912 - val_loss: 0.6704
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6609 - loss: 0.6274 - val_accuracy: 0.6646 - val_loss: 0.6033
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7886 - loss: 0.4732 - val_accuracy: 0.6468 - val_loss: 0.6241
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.7490 - loss: 0.5116 - val_accuracy: 0.7546 - val_loss: 0.5298
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8494 - loss: 0.3608 - val_accuracy: 0.7948 - val_loss: 0.4791


In [ ]:
 #7. Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("\nTest Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7923 - loss: 0.4776

Test Loss: 0.47262585163116455
Test Accuracy: 0.7957199811935425


In [ ]:
# 8. Make Prediction on New Review
# Load word index
word_index = imdb.get_word_index()

def encode_review(text):
    """
    Encode text the same way IMDB was trained
    """
    encoded = [1]  # start token

    for word in text.lower().split():
        idx = word_index.get(word)
        if idx is not None and idx < VOCAB_SIZE:
            encoded.append(idx + 3)
        else:
            encoded.append(2)  # unknown word

    return pad_sequences([encoded], maxlen=MAX_LEN)
# Example review
reviews = [
    "this movie was good and enjoyable",
    "this movie was bad and boring",
    "i loved this film it was amazing",
    "i hated this movie it was terrible"
]

for review in reviews:
    score = model.predict(encode_review(review))[0][0]
    print("\nReview:", review)
    print("Sentiment Score:", score)
    print("Prediction:", "Positive 😊" if score > 0.5 else "Negative 😞")

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step

Review: this movie was good and enjoyable
Sentiment Score: 0.96463615
Prediction: Positive 😊
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

Review: this movie was bad and boring
Sentiment Score: 0.23133437
Prediction: Negative 😞
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Review: i loved this film it was amazing
Sentiment Score: 0.9871753
Prediction: Positive 😊
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

Review: i hated this movie it was terrible
Sentiment Score: 0.27703738
Prediction: Negative 😞
